# Information Visualization Altair Demo 5
School of Information, University of Michigan


## Plan
1. Review
2. Export 

## Resources/References 
*  [Export documentation](https://altair-viz.github.io/user_guide/saving_charts.html?highlight=html#saving-altair-charts)
* [idyll example](https://idyll-lang.github.io/examples/csv/)
* [vega example](https://vega.github.io/vega-lite/tutorials/streaming.html)







## 1. Basic Chart

In [0]:
# imports we will use
import altair as alt
import pandas as pd
from vega_datasets import data as vega_data

# usa = vega_data.us_10m.url

# states = alt.topo_feature(vega_data.us_10m.url, 'states')
# capitals = vega_data.us_state_capitals.url

# # US states background
# background = alt.Chart(states).mark_geoshape(
#     fill='lightgray',
#     stroke='white'
# ).properties(
#     title='US State Capitols',
#     width=650,
#     height=400
# ).project('albersUsa')


In [0]:
## overlay election data onto map 

primaryData=pd.read_csv("https://raw.githubusercontent.com/raubsaurier/si649_labs/master/lab5_altair/primaryData.csv")


# alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
#     stroke='#aaa', strokeWidth=0.25
# ).transform_lookup(
#     lookup='state', from_=alt.LookupData(data=test, key='state', fields=['trump_lead'])
# ).encode(
#     alt.Color('trump_lead:Q',
#               scale=alt.Scale(domain=[0, 0.3], clamp=True), 
#               legend=alt.Legend(format='%')),
#     alt.Tooltip('trump_lead:Q', format='.0%')
# )

primaryData['end_date'] = pd.to_datetime(primaryData['end_date'])
primaryData['created_at_date'] = pd.to_datetime(primaryData['created_at_date'])
states = {
         'Alaska':'AK' ,
        'Alabama':'AL',
        'Arkansas':'AR',
        'Arizona':'AZ' ,
        'California':'CA',
        'Colorado':'CO',
        'Connecticut':'CT',
        'District of Columbia':'DC',
        'Delaware': 'DE',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Guam': 'GU',
        'Hawaii':'HI',
        'Iowa':'IA',
        'Idaho':'ID',
        'Illinois':'IL',
        'Indiana':'IN',
        'Kansas':'KS',
        'Kentucky':'KY',
        'Louisiana':'LA',
        'Massachusetts':'MA',
        'Maryland':'MD',
        'Maine':'ME',
        'Michigan':'MI',
        'Minnesota':'MN',
        'Missouri':'MO',
        'Mississippi':'MS',
        'Montana':'MT',
        'North Carolina':'NC',
        'North Dakota':'ND',
        'Nebraska':'NE',
        'National': 'NA',
        'New Hampshire':'NH',
        'New Jersey':'NJ',
        'New Mexico':'NM',
        'Nevada':'NV',
        'New York':'NY',
        'Ohio':'OH',
        'Oklahoma':'OK',
        'Oregon':'OR',
        'Pennsylvania':'PA',
        'Puerto Rico':'PR',
        'Rhode Island':'RI',
        'South Carolina':'SC',
        'South Dakota':'SD',
        'Tennessee':'TN',
        'Texas':'TX',
        'Utah':'UT',
        'Virginia':'VA',
        'Vermont':'VT',
        'Washington':'WA',
        'Wisconsin':'WI',
        'West Virginia':'WV',
        'Wyoming':'WY' 
}

## map states to regions 
regions = {
        'AK': 'Other',
        'AL': 'South',
        'AR': 'South',
        'AZ': 'West',
        'CA': 'West Coast',
        'CO': 'West',
        'CT': 'North',
        'DC': 'South',
        'DE': 'North',
        'FL': 'South',
        'GA': 'South',
        'HI': 'Other',
        'IA': 'Midwest',
        'ID': 'West',
        'IL': 'Midwest',
        'IN': 'Midwest',
        'KS': 'Midwest',
        'KY': 'South',
        'LA': 'South',
        'MA': 'North',
        'MD': 'Mid-Atlantic',
        'ME': 'North',
        'MI': 'Midwest',
        'MN': 'Midwest',
        'MO': 'Midwest',
        'MS': 'South',
        'MT': 'West',
        'NA': 'National',
        'NC': 'South',
        'ND': 'Midwest',
        'NE': 'West',
        'NH': 'North',
        'NJ': 'North',
        'NM': 'West',
        'NV': 'West',
        'NY': 'North',
        'OH': 'Midwest',
        'OK': 'South',
        'OR': 'West Coast',
        'PA': 'North',
        'PR': 'Other',
        'RI': 'North',
        'SC': 'South',
        'SD': 'Midwest',
        'TN': 'South',
        'TX': 'West',
        'UT': 'West',
        'VA': 'South',
        'VT': 'North',
        'WA': 'West Coast',
        'WI': 'Midwest',
        'WV': 'South',
        'WY': 'West'
}

## map states to regions 
primary_method = {
        'Automated Phone': 'Phone',
        'Online': 'Online',
        'Online/IVR': 'Hybrid',
        'Online/Text': 'Hybrid',
        'IVR/Live Phone': 'Hybrid',
        'IVR/Text': 'Hybrid',
        'IVR/Online': 'Hybrid',
        'Live Phone/Online': 'Hybrid',
        'Live Phone/Text': 'Hybrid',
        'Live Phone': 'Phone',
}

primaryData['state'].fillna('National', inplace=True)
## replace candidate names with their more common names
primaryData = primaryData.replace('Bernard Sanders', 'Bernie Sanders')
primaryData = primaryData.replace('Joseph R. Biden Jr.', 'Joe Biden')
primaryData = primaryData.replace('Maine CD-1', 'Maine')
primaryData = primaryData.replace('Maine CD-2', 'Maine')
primaryData['state_abbrev'] = primaryData['state'].map(states)
primaryData['region'] = primaryData['state_abbrev'].map(regions)
primaryData['method_abbrev'] = primaryData['methodology'].map(primary_method)


In [0]:
primaryData.to_csv('primaryData.csv')

In [0]:
## count how many times each candidate won in h2h matchups: 

def makeC3(data):

  ## calculate percentage of wins:
  g = data.groupby(['candidate_name', 'state'])['winner']
  percentage_wins = g.aggregate(lambda x: (x == 'YES').sum()) / g.size()

  data=data.join(percentage_wins, on=['candidate_name', 'state'], how='left', rsuffix='_percentage')
    
  domain = ["Bernie Sanders", "Pete Buttigieg", "Elizabeth Warren", "Joe Biden"]
  data= data[data['candidate_name'].isin(domain)]



  selectCandidate=alt.selection_single(
      fields=["candidate_name"],#where our selection is based on
      bind=alt.binding_select(
          options=domain, 
          name="Select Candidate") #drop down menu 
  )

  c0 =alt.Chart(data).mark_bar().transform_filter(
          (alt.datum.state=="Iowa")
    ).encode(
      alt.X('candidate_name:N', 
            title=""),
      alt.Y('winner_percentage:Q',
            axis=alt.Axis(format='%'),
            scale=alt.Scale(domain=(0,1)),
            title='% of Times Candidate Beat Trump'),
      alt.Color('candidate_name:N')
      ).add_selection(selectCandidate).encode(opacity=alt.condition(selectCandidate,
                                    alt.value(1),
                                    alt.value(0)
                                    )
        ).properties(title="Iowa")

  c1 =alt.Chart(data).mark_bar().transform_filter(
          (alt.datum.state=="New Hampshire")
    ).encode(
      alt.X('candidate_name:N', 
            title=""),
      alt.Y('winner_percentage:Q',
            axis=alt.Axis(format='%'),
            title=''),
      alt.Color('candidate_name:N')
      ).add_selection(selectCandidate).encode(opacity=alt.condition(selectCandidate,
                                    alt.value(1),
                                    alt.value(0)
                                    )
        ).properties(title="New Hampshire")

  c2=alt.Chart(data).mark_bar().transform_filter(
          (alt.datum.state=="Nevada")
    ).encode(
      alt.X('candidate_name:N', 
            title=""),
      alt.Y('winner_percentage:Q',
            title='',
            axis=alt.Axis(format='%'),
            scale=alt.Scale(domain=(0, 1))),
      alt.Color('candidate_name:N')
      ).add_selection(selectCandidate).encode(opacity=alt.condition(selectCandidate,
                                    alt.value(1),
                                    alt.value(0)
                                    )
        ).properties(title="Nevada")
    
  c3=alt.Chart(data).mark_bar().transform_filter(
          (alt.datum.state=="National")
    ).encode(
      alt.X('candidate_name:N', 
            title=""),
      alt.Y('winner_percentage:Q',
            axis=alt.Axis(format='%'),
            title=''),
             alt.Color('candidate_name:N', title='Dem. Candidate')).add_selection(selectCandidate).encode(opacity=alt.condition(selectCandidate,
                                    alt.value(1),
                                    alt.value(0)
                                    )
        ).properties(title="National")
  #return c1
  
  return alt.hconcat(c0|c1|c2|c3).resolve_axis("shared").properties(title="Democratic Candidates' Average Favorability (vs. Donald Trump)")


c3 = makeC3(primaryData)
c3


## TO DO: make sure that this explicitly says compared to Trump! 

## how many times do they beat trump? (group by end_date)
### add error bars 
c3.save('nh_nv_national_chart.json')


In [0]:
### does the method of polling show different electability among the Dem candidates? 
def makeC4(data):
  # make the first chart. The size of the point
  # will change based on the slider

  
  domain = ["Bernie Sanders", "Pete Buttigieg", "Elizabeth Warren", "Joe Biden"]
  data= data[data['candidate_name'].isin(domain)]

  data =data[~data['methodology'].isnull()]
  data =data[data['region'].notna()]

  methods = list(data["region"].unique())


  selectRegion=alt.selection_single(
      fields=["region"],#where our selection is based on
      bind=alt.binding_select(
          options=methods, 
          name="Select Region") #drop down menu 
  )

  c1=alt.Chart(data).mark_point(filled=True).transform_filter(
      alt.FieldOneOfPredicate(field='candidate_name', oneOf=['Bernie Sanders', 'Elizabeth Warren'])
                                                              ).transform_joinaggregate(
      mean_pct = 'mean(pct)',
      groupby=['region', 'methodology']
  ).encode(
      alt.X('methodology', title=""),
      alt.Y('mean_pct:Q', title="Average Polling % Results"),
      alt.Color('region:N',title="Region"),
      column=alt.Column('candidate_name:N', title="Democratic Candidate"),
      tooltip=['pollster:N', 'sample_size']
      ).add_selection(selectRegion).encode(opacity=alt.condition(selectRegion,
                                  alt.value(1),
                                  alt.value(0)
                                  )
      ).properties(title="Average Poll Results % by Method and U.S. Region", 
                   height=150, 
                   width=200).interactive()
      
  c2=alt.Chart(data).mark_point(filled=True).transform_filter(
      alt.FieldOneOfPredicate(field='candidate_name', oneOf=['Joe Biden', 'Pete Buttigieg'])
                                                              ).transform_joinaggregate(
      mean_pct = 'mean(pct)',
      groupby=['region', 'methodology']
  ).encode(
      alt.X('methodology', title="Polling Method"),
      alt.Y('mean_pct:Q', title="Average Polling % Results"),
      alt.Color('region:N',title="Region"),
      column=alt.Column('candidate_name:N', title=""),
      tooltip=['pollster:N', 'sample_size']
      ).add_selection(selectRegion).encode(opacity=alt.condition(selectRegion,
                                  alt.value(1),
                                  alt.value(0)
                                  )
      ).properties(height=150, 
                   width=200).interactive()

  return c1& c2
  # c2 =alt.Chart(data).mark_line().transform_filter(
  #           (alt.datum.state=="New Hampshire")
  #     ).encode(
  #       x='end_date:T',
  #       y='mean(pct):Q',
  #       color="candidate_name:N").add_selection(selectCandidate).encode(opacity=alt.condition(selectCandidate,
  #                                   alt.value(1),
  #                                   alt.value(0)
  #                                   )
  #       ).properties(title="Polling Average % General Election Support in New Hampshire")

  # return c1|c2
    

c4 = makeC4(primaryData)
c4
c4.save('candidate_by_poll_method.json')
## to do - group colors by IVR/Live Phone/Online instead of the sub methods


In [0]:
#1.3 get c3

#       )
### does the method of polling show different electability among the Dem candidates? 
def makeC5(data):
#   # make the first chart. The size of the point
#   # will change based on the slider

  domain = ["Donald Trump"]

  data= data[data['candidate_name'].isin(domain)].drop_duplicates()
  data =data[data['region'].notna()]

  data['national_poll'] = data['region'].apply(lambda x: 'Yes' if x == 'National' else 'No')

  ## set up region selection menu: 
  regions = list(data['region'].unique())
  regions.remove('National')
  
  sortRegions = ['National','North','Midwest','South','West', 'West Coast','Other']

  palette = alt.Scale(domain=sortRegions,
                  range=['#adadad', '#008b8b', '#b846b8','#ed64b6','olive', 'green','orange'])
  


  selectRegion=alt.selection_single(
        fields=["region"],#where our selection is based on
        bind=alt.binding_select(
            options=regions, 
            name="Select Region") #drop down menu 
    )
  

  c = alt.Chart(data).transform_joinaggregate(
      mean_pct = 'mean(pct)',
      groupby=['pollster', 'state', 'end_date']
  )
  c0 = c.mark_errorbar(extent='stdev').encode(
      alt.X('end_date:T', title="Poll End Date"),
      y='mean_pct:Q',
      color='region:N'
      )


  c1 = c.transform_filter(alt.datum.region!="National").mark_circle().encode(
      alt.X('end_date:T', title="Poll End Date"),
      alt.Y('mean_pct:Q', title="Mean % Favorability"),
      alt.Color('region:N', scale=palette,
                sort=sortRegions, title="U.S. Region"),
       tooltip=['state:N','pollster:N']
      ).add_selection(selectRegion).encode(opacity=alt.condition(selectRegion,
                                    alt.value(1),
                                    alt.value(0)
                                    )
        ).properties(
          title="Donald Trump's General Election Favorability by Region")
  
  ## set up radio for national polls
  national_domain = list(data['national_poll'].unique())
  selectNational =alt.selection_single(
    fields=['national_poll'],
    init={
        "national_poll":national_domain[0]
        },
    bind=alt.binding_radio(
        options=national_domain,
        name="Show National Results?"
    )
  )

  c2 = alt.Chart(data).mark_circle().transform_filter(alt.datum.state=="National").transform_joinaggregate(
        mean_pct = 'mean(pct)',
        std_pct = 'stdev(pct)',
        groupby=['pollster', 'end_date']
    ).encode(
        alt.X('yearmonthdate(end_date):T', title="Poll End Date"),
        alt.Y('mean_pct:Q',title=""),
        alt.Color('region:N', scale=alt.Scale(range=['red'])),
        tooltip=['state:N','pollster:N']
        ).add_selection(selectNational).encode(opacity=alt.condition(selectNational,
                                      alt.value(1),
                                      alt.value(0)
                                      )
          ).properties(
            title="Donald Trump's General Election Favorability by Region").interactive()

   
      
  return c1+c2

#   #states = list(data["state"].unique()

# # #step2
#   c1=alt.Chart(data).mark_point(filled=True).encode(
#       x='end_date:T',
#       y='pct:Q',
#       color='region:N',
#       ).properties(title="General Election Favorability by Region")
      
#   

## TO DO: get average by poll_id and then create error bars 


c5 = makeC5(primaryData)
c5
c5.save('trump_regional_favorability.json')

## 2. Export to images
export to png and svg 

In [0]:
#2.1 can export to ['png', 'svg']
c3.save("chart.png")
c3.save("chart.svg")


If you have errors related to selenium and chromedriver, run the following command and solve the "version of Google Chrome and Chromium Browser don't match" error  

In [0]:
# 2.2 
# apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 25 not upgraded.
Need to get 72.7 MB of archives.
After this operation, 259 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 79.0.3945.130-0ubuntu0.18.04.1 [1,078 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 79.0.39

## 3.Export to json 

In [0]:
#3.1
c3.to_json()

'{\n  "$schema": "https://vega.github.io/schema/vega-lite/v4.0.2.json",\n  "config": {\n    "view": {\n      "continuousHeight": 300,\n      "continuousWidth": 400\n    }\n  },\n  "data": {\n    "name": "data-8e25068d8b9a578f41a59c91e539d690"\n  },\n  "datasets": {\n    "data-8e25068d8b9a578f41a59c91e539d690": [\n      {\n        "Acceleration": 12.0,\n        "Cylinders": 8,\n        "Displacement": 307.0,\n        "Horsepower": 130.0,\n        "Miles_per_Gallon": 18.0,\n        "Name": "chevrolet chevelle malibu",\n        "Origin": "USA",\n        "Weight_in_lbs": 3504,\n        "Year": "1970-01-01"\n      },\n      {\n        "Acceleration": 11.5,\n        "Cylinders": 8,\n        "Displacement": 350.0,\n        "Horsepower": 165.0,\n        "Miles_per_Gallon": 15.0,\n        "Name": "buick skylark 320",\n        "Origin": "USA",\n        "Weight_in_lbs": 3693,\n        "Year": "1970-01-01"\n      },\n      {\n        "Acceleration": 11.0,\n        "Cylinders": 8,\n        "Displac

In [0]:
#3.2
c3.save('chart.json')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

[json beautifier ](https://jsonformatter.org/)

### 3.3 Shorter JSON: Uinsg url 


In [0]:
#3.3.1 
car_url = vega_data.cars.url
car_url

'https://vega.github.io/vega-datasets/data/cars.json'

In [0]:
#3.3.2
hp_mpg_2=alt.Chart(car_url).mark_circle(size=80,opacity=0.5).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color="Origin:N"
)
hp_mpg_2

alt.Chart(...)

In [0]:
#3.3.3
c3=makeC3(car_url)
c3.save("c3.json")
c3

alt.HConcatChart(...)

In [0]:
print(c3.to_json())

{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.0.2.json",
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 400
    }
  },
  "data": {
    "url": "https://vega.github.io/vega-datasets/data/cars.json"
  },
  "hconcat": [
    {
      "encoding": {
        "color": {
          "field": "Origin",
          "type": "nominal"
        },
        "size": {
          "condition": {
            "test": "(datum.Horsepower < selector001.cutoff)",
            "value": 300
          },
          "value": 10
        },
        "x": {
          "field": "Horsepower",
          "type": "quantitative"
        },
        "y": {
          "field": "Miles_per_Gallon",
          "type": "quantitative"
        }
      },
      "mark": {
        "opacity": 0.5,
        "size": 80,
        "type": "circle"
      },
      "selection": {
        "selector001": {
          "bind": {
            "input": "range",
            "max": 230.0,
            "min": 46.0,
     

In [0]:
c4=makeC4(car_url)
c4.save("c4.json")
c4

alt.Chart(...)

## 4.Display with [VegaEmbed](https://github.com/vega/vega-embed)
* copy-paste json 
* link to online json file 
* link to local json file (you need to run server)

> cd  *\[directory where your .html is stored\]*

>  python3 *(or python)* -m http.server


#### Useful embed config feature 

* [Renderer](https://vega.github.io/vega/docs/api/view/#view_renderer)
* [Theme](https://github.com/vega/vega-themes) 

